In [1]:
# !unzip robohearts.zip
#%cd robohearts/

In [1]:
import gym
import multiprocessing
from gymhearts.Hearts import *
from gymhearts.Agent.agent_random import RandomAgent
from gymhearts.Agent.agent_mc_simple import MonteCarlo
from gymhearts.Agent.utils_env import *
from gymhearts.Agent.utils_nn import *
from tqdm import tqdm_notebook

In [2]:
# ----------- TRAINING CONFIGURATION -------------
# Number of games to train on 
TRAINING_ITERS = 1000

# Number of episodes to run during model evaluation
NUM_EPISODES = 10

# Number of model evaluations to average together
NUM_TESTS = 10

# Max score for players to win the game
MAX_SCORE = 100

# Set to false to skip training
run_train = False

# Set to true to resume training from an existing model
continue_train = False

# Run testing on a random agent for comparison
run_random = False

# Name of the file that is saved :: {model_name}.th
model_name = 'mc_simple'

# Configuration parameters for the model
mc_simple_config = {
    'print_info' : False,
    'epsilon' : 0.01,
    'gamma' : 1,
    'alpha': 0.001
}

if continue_train:
    mc_simple_config['load_model'] = model_name

playersNameList = ['MonteCarlo', 'Rando', 'Randy', 'Randall']
agent_list = [0, 0, 0, 0]

agent_list[0] = MonteCarlo(playersNameList[0], mc_simple_config)
agent_list[1] = RandomAgent(playersNameList[1], {'print_info' : False})
agent_list[2] = RandomAgent(playersNameList[2], {'print_info' : False})
agent_list[3] = RandomAgent(playersNameList[3], {'print_info' : False})

In [4]:
# ------- TRAIN MONTE CARLO SIMPLE AGENT ---------
env = gym.make('Hearts_Card_Game-v0')
env.__init__(playersNameList, MAX_SCORE)
weights = []
if run_train:
    for trn_episode in tqdm_notebook(range(TRAINING_ITERS)):
        # Save the model every 50 steps
        if trn_episode % 50 == 0:
            save_model(weights, model_name, 'mc_simple')
        observation = env.reset()
        history = []
        while True:
            #env.render()
            now_event = observation['event_name']
            IsBroadcast = observation['broadcast']
            action = None
            if IsBroadcast == True:
                for agent in agent_list:
                    agent.Do_Action(observation)
            else:
                playName = observation['data']['playerName']
                for agent in agent_list:
                    if agent.name == playName:
                        action = agent.Do_Action(observation)

            # update my agent
            if now_event == 'RoundEnd':
                errors = agent_list[0].update_weights(history, -reward['MonteCarlo'])
                history = []
                weights = agent_list[0].weight_vec
            if now_event == 'GameOver':
                  break
            if not IsBroadcast and observation['data']['playerName'] == 'MonteCarlo':
                history.append(observation)
            observation, reward, done, info = env.step(action)
    save_model(weights, model_name, 'mc_simple')

In [ ]:
# Function to test mc simple model with multiprocessing
def run_test(num_won):
    # Weird hack to make progress bars render properly
    print(' ', end='', flush=True)
    for i_ep in tqdm_notebook(range(NUM_EPISODES)):
        observation = env.reset()
        while True:
            now_event = observation['event_name']
            IsBroadcast = observation['broadcast']
            action = None
            if IsBroadcast == True:
                for agent in agent_list:
                    agent.Do_Action(observation)
            else:
                playName = observation['data']['playerName']
                for agent in agent_list:
                    if agent.name == playName:
                        action = agent.Do_Action(observation)
            if now_event == 'GameOver':
                num_won += int(observation['data']['Winner'] == 'MonteCarlo')
                break
            observation, reward, done, info = env.step(action)
    return num_won

In [3]:
# ---------- EVALUATE MC SIMPLE AGENT --------------
    
env = gym.make('Hearts_Card_Game-v0')
env.__init__(playersNameList, MAX_SCORE)

# Evaluation parameters for testing
mc_simple_config = {
    'print_info' : False,
    'load_model' : model_name
}

agent_list[0] = MonteCarlo(playersNameList[0], mc_simple_config)
mc_wins = [0] * NUM_TESTS
   
pool = multiprocessing.Pool(processes=NUM_TESTS)
mc_wins = pool.map(run_test, mc_wins)
print(mc_wins)
pool.close()
pool.join()


























[248, 262, 257, 248, 223, 225, 250, 260, 257, 247, 234, 237, 215, 237, 250, 256, 223, 222, 236, 232, 236, 238, 227, 210, 228]


In [ ]:
# Function to test random model with multiprocessing
def run_test_random(num_won):
    # Weird hack to make progress bars render properly
    print(' ', end='', flush=True)
    for i_ep in tqdm_notebook(range(NUM_EPISODES)):
        observation = env.reset()
        while True:
            now_event = observation['event_name']
            IsBroadcast = observation['broadcast']
            action = None
            if IsBroadcast == True:
                for agent in agent_list:
                    agent.Do_Action(observation)
            else:
                playName = observation['data']['playerName']
                for agent in agent_list:
                    if agent.name == playName:
                        action = agent.Do_Action(observation)
            if now_event == 'GameOver':
                num_won += int(observation['data']['Winner'] == 'Randman')
                break
            observation, reward, done, info = env.step(action)
    return num_won

In [25]:
# ----------- EVALUTATE RANDOM AGENT ---------------
if run_random:
    env = gym.make('Hearts_Card_Game-v0')
    env.__init__(playersNameList, MAX_SCORE)
    playersNameList[0] = 'Randman'
    agent_list[0] = RandomAgent(playersNameList[0])
    rand_wins = [0] * NUM_TESTS
    pool = multiprocessing.Pool(processes=NUM_TESTS)
    rand_wins = pool.map(run_test, rand_wins)
    print(rand_wins)
    pool.close()
    pool.join()

[244, 278, 237, 258, 237, 253, 258, 275, 274, 248]


In [4]:
print(f"Monte Carlo won {sum(mc_wins)/len(mc_wins)} times on average :: {str(mc_wins)}")
for wins in mc_wins:
    print(wins)
if run_random:
    print(f"Random won {sum(rand_wins)/len(rand_wins)} times on average :: {str(rand_wins)}")

Monte Carlo won 238.32 times on average :: [248, 262, 257, 248, 223, 225, 250, 260, 257, 247, 234, 237, 215, 237, 250, 256, 223, 222, 236, 232, 236, 238, 227, 210, 228]
248
262
257
248
223
225
250
260
257
247
234
237
215
237
250
256
223
222
236
232
236
238
227
210
228


The Monte Carlo weights are: [2.576861053203586, 0.0746679771208893, -1.0812505668744499, -0.7988069051959343, -0.18287651208895211, -1.3484899819506349, -0.7738220160521773, -0.5906825585410745, -0.8971680184937948, -0.885565000008097, -1.3417259322269985, 0.2535853856420063, -1.441329568781756, -1.626241953689169, -0.4520713221939589, -0.6678954038095386, -2.25007892981782, -2.431379022777693, -1.567284905178469, -0.490646940611233, 0.34596233132385223, -1.3335021078350033, -0.7279177782535996, -1.779761417596184, -0.05179903215749995, -1.5942588118086698, -0.7143773868101742, -0.41782520640881626, -0.6012057234741566, -0.1547484832852079, -1.3914770806800125, -1.698230349430356, -0.524848250080739, -1.539843493071182